In [2]:
!pip install pyspark >> None

In [25]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import sum, desc, col

In [4]:
spark = SparkSession.builder.appName("BookSales").getOrCreate()

— Используя Spark прочитайте данные из файла csv.


In [14]:
csv_url = "https://docs.google.com/spreadsheets/d/1ERqHT7q3k0POkyqnJ5_GaM93-BVWHV3HeB7g1ub2lbE/export?format=csv"

In [15]:
books_df = pd.read_csv(csv_url)
books_df.to_csv('books.csv', index=False)

In [16]:
books_df = spark.read.csv('books.csv', header=True, inferSchema=True)

— Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров.

In [17]:
filtered_books_df = books_df.filter("sales > 3000")

In [18]:
filtered_books_df.show()

+--------------------+-------------------+---------------+-----+----+
|               title|             author|          genre|sales|year|
+--------------------+-------------------+---------------+-----+----+
|                1984|      George Orwell|Science Fiction| 5000|1949|
|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
+--------------------+-------------------+---------------+-----+----+



— Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.

In [20]:
sales_by_genre = books_df.groupBy("genre").sum("sales")

In [26]:
sales_by_genre = sales_by_genre.withColumnRenamed("sum(sales)", "total_sales")

In [27]:
sales_by_genre.show()

+---------------+-----------+
|          genre|total_sales|
+---------------+-----------+
|Southern Gothic|       4000|
|          Novel|       6500|
|        Fantasy|       3000|
|Science Fiction|       5000|
+---------------+-----------+



— Отсортируйте данные по общему объему продаж в порядке убывания.

In [31]:
sorted_books_df = books_df.orderBy(desc("sales"))

In [32]:
sorted_books_df.show()

+--------------------+-------------------+---------------+-----+----+
|               title|             author|          genre|sales|year|
+--------------------+-------------------+---------------+-----+----+
|                1984|      George Orwell|Science Fiction| 5000|1949|
|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|The Lord of the R...|     J.R.R. Tolkien|        Fantasy| 3000|1954|
|The Catcher in th...|      J.D. Salinger|          Novel| 2000|1951|
+--------------------+-------------------+---------------+-----+----+

